In [4]:
# Libraries

import pandas as pd
import finviz # Finviz API to get Stock data
from finviz.screener import Screener # Finviz API to get stock screener information
from pymongo import MongoClient # MongoClient to connect to MongoDB
from pprint import pprint


In [5]:
class Finviz:
    def __init__(self,host,port):
        self.client = MongoClient(host,port)
        self.db = self.client.numeraxial
        self.collections = self.db.list_collection_names()
        self.collection_name = self.db.stock
        
    def get_all_stock(self):
        self.stock_list = Screener()
        self.stock_list.to_csv('finviz_stocks_list.csv')
    def get_all_ticker(self):
        self.df = pd.read_csv("finviz_stocks_list.csv")
        self.tickers = self.df['Ticker'].tolist()
        self.companies = self.df['Company'].tolist()
    def get_stock_info(self, ticker, index):
        stock = finviz.get_stock(ticker)
        entry={}
        entry['Symbol'] = ticker
        entry['Name'] = self.companies[index]
        entry['Sector'] = stock['Sector']
        entry['Industry'] = stock['Industry']
        entry['Price'] = stock['Price']
        entry['Prev Close'] = stock['Prev Close']
        entry['Change'] = stock['Change']
        entry['Market Cap'] = stock['Market Cap']
        entry['P/E'] = stock['P/E']
        entry['Volume'] = stock['Volume']
        entry['ROE'] = stock['ROE']
        entry['ROA'] = stock['ROA']
        entry['Dividend'] = stock['Dividend']
        entry['EPS this Y'] = stock['EPS this Y']
        entry['52W High'] = stock['52W High']
        entry['52W Low'] = stock['52W Low']
        return entry
    def get_stocks_info(self):
        self.get_all_stock()
        self.get_all_ticker()
        for index in range(len(self.tickers)):
            entry = self.get_stock_info(self.tickers[index], index)
            self.collection_name.update_one({'Symbol': entry['Symbol']}, {"$set": entry}, True)
    
        
        

In [6]:
fz = Finviz('localhost',27017)
fz.get_stocks_info()

100%|██████████| 414/414 [02:02<00:00,  3.38it/s]
